# Chapter 13: 모델 평가 및 테스트

## 1. 학습 목표

* **정량적 지표**: Perplexity, BLEU, ROUGE 등의 자동 평가 지표를 이해하고 계산한다.
* **정성적 평가**: GPT-4 등 고성능 모델을 심판(Judge)으로 활용하는 **LLM-as-Judge** 시스템을 구축한다.
* **기능 평가**: 모델이 명령어를 잘 따르는지(Instruction Following) 형식을 검증한다.
* **안전장치**: 재앙적 망각(Catastrophic Forgetting) 테스트를 통해 모델의 품질을 보장한다.

## 2. 자동 평가 지표 (Quantitative Metrics)

### 2.1 주요 지표 개요

사람이 일일이 채점하는 것은 비용이 많이 들기 때문에, 자동화된 수치 지표를 먼저 확인한다.

| 지표 | 용도 | 설명 | 해석 |
| --- | --- | --- | --- |
| **Perplexity (PPL)** | 언어 모델링 | 모델이 다음에 올 단어를 얼마나 확신을 가지고 예측하는가? | **낮을수록 좋음** (1에 가까울수록 완벽) |
| **BLEU** | 번역, 생성 | 정답 문장과 생성 문장의 N-gram(단어 조합)이 얼마나 겹치는가? | 높을수록 좋음 (보통 0~100점) |
| **ROUGE** | 요약 | 생성된 요약문이 정답 요약문의 핵심 단어를 얼마나 포함(Recall)하는가? | 높을수록 좋음 |
| **Accuracy** | 객관식/분류 | 정해진 답을 정확히 맞췄는가? | 높을수록 좋음 |

### 2.2 Perplexity 계산 실습

Perplexity는 모델의 '놀람' 정도를 의미한다. 모델이 텍스트를 자연스럽게 예측할수록 수치가 낮아진다.

In [3]:
%pip install -q evaluate rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import math
from transformers import AutoModelForCausalLM, AutoTokenizer

def calculate_perplexity(model, tokenizer, text):
    """
    주어진 텍스트에 대한 모델의 Perplexity를 계산하는 함수다.
    """
    # 텍스트를 토큰화 (입력을 텐서로 변환)
    encodings = tokenizer(text, return_tensors="pt")
    input_ids = encodings.input_ids.to(model.device)

    # 모델의 손실(Loss) 계산
    # labels를 input_ids와 동일하게 주면 언어 모델링 손실(Next Token Prediction Loss)이 계산된다.
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss

    # Perplexity = exp(Loss)
    perplexity = math.exp(loss.item())
    return perplexity

# 모델 로드 (이전 챕터에서 학습된 모델 또는 베이스 모델 사용)
model_id = "Qwen/Qwen3-14B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16
)

# 테스트
text = "인공지능은 컴퓨터 공학의 중요한 분야이다."
ppl = calculate_perplexity(model, tokenizer, text)
print(f"문장: {text}")
print(f"Perplexity: {ppl:.2f}")

/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 8/8 [00:06<00:00,  1.26it/s]


문장: 인공지능은 컴퓨터 공학의 중요한 분야이다.
Perplexity: 13.17


### 2.3 ROUGE & BLEU 계산

생성된 텍스트와 정답(Reference) 간의 유사도를 측정한다. `evaluate` 라이브러리를 사용하면 간편하게 계산할 수 있다.

In [4]:
# 라이브러리 설치 필요: pip install evaluate rouge_score
import evaluate

def evaluate_metrics(predictions, references):
    """
    ROUGE와 BLEU 점수를 동시에 계산하는 함수다.
    """
    # 지표 로드
    rouge = evaluate.load('rouge')
    bleu = evaluate.load('bleu')

    # ROUGE 계산
    rouge_results = rouge.compute(predictions=predictions, references=references)

    # BLEU 계산
    bleu_results = bleu.compute(predictions=predictions, references=references)

    return {
        "rouge1": rouge_results['rouge1'],
        "rouge2": rouge_results['rouge2'],
        "rougeL": rouge_results['rougeL'],
        "bleu": bleu_results['bleu']
    }

# 예시 데이터
preds = ["딥러닝은 머신러닝의 한 분야이다."]
refs = ["딥러닝은 인공지능의 하위 분야인 머신러닝의 일종이다."]

metrics = evaluate_metrics(preds, refs)
print("평가 결과:", metrics)

평가 결과: {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'bleu': 0.0}


## 3. LLM-as-Judge (정성적 평가)

텍스트 생성 모델의 품질은 수치로만 판단하기 어렵다. 최근에는 GPT-4와 같은 고성능 모델에게 "이 답변이 얼마나 좋은지 평가해줘"라고 요청하는 **LLM-as-Judge** 방식이 표준으로 자리 잡고 있다.

### 3.1 평가 프롬프트 설계

심판 모델에게 명확한 채점 기준(Rubric)을 제공해야 공정한 평가가 가능하다.

In [5]:
def llm_judge_prompt(question, response):
    """
    심판 모델(GPT-4 등)에게 입력할 평가용 프롬프트를 생성한다.
    """
    prompt = f"""당신은 공정한 AI 평가자입니다. 다음 질문에 대한 AI의 응답을 평가해주세요.

[질문]: {question}
[응답]: {response}

다음 4가지 기준으로 1점부터 10점까지 점수를 매기고, 그 이유를 설명해주세요.

1. 정확성(Accuracy): 정보가 사실과 일치하는가?
2. 완전성(Completeness): 질문의 모든 요구사항을 충족했는가?
3. 명확성(Clarity): 문장이 이해하기 쉽고 논리적인가?
4. 유용성(Usefulness): 사용자에게 실질적인 도움이 되는가?

반드시 아래 JSON 형식으로만 출력하세요:
{{
    "accuracy": <점수>,
    "completeness": <점수>,
    "clarity": <점수>,
    "usefulness": <점수>,
    "overall_score": <평균 점수>,
    "feedback": "<구체적인 피드백>"
}}"""
    return prompt

# 사용 예시 (실제로는 OpenAI API 등을 호출하여 이 프롬프트를 보낸다)
q = "파이썬의 GIL에 대해 설명해줘."
a = "GIL은 Global Interpreter Lock의 약자로..."
print(llm_judge_prompt(q, a))

당신은 공정한 AI 평가자입니다. 다음 질문에 대한 AI의 응답을 평가해주세요.

[질문]: 파이썬의 GIL에 대해 설명해줘.
[응답]: GIL은 Global Interpreter Lock의 약자로...

다음 4가지 기준으로 1점부터 10점까지 점수를 매기고, 그 이유를 설명해주세요.

1. 정확성(Accuracy): 정보가 사실과 일치하는가?
2. 완전성(Completeness): 질문의 모든 요구사항을 충족했는가?
3. 명확성(Clarity): 문장이 이해하기 쉽고 논리적인가?
4. 유용성(Usefulness): 사용자에게 실질적인 도움이 되는가?

반드시 아래 JSON 형식으로만 출력하세요:
{
    "accuracy": <점수>,
    "completeness": <점수>,
    "clarity": <점수>,
    "usefulness": <점수>,
    "overall_score": <평균 점수>,
    "feedback": "<구체적인 피드백>"
}


## 4. 태스크별 기능 평가 (Instruction Following)

모델이 지시사항(특히 포맷)을 잘 지키는지 검증하는 코드다. 예를 들어 "JSON으로 출력하라"는 지시를 어기면, 내용은 맞아도 시스템 연동에 실패하기 때문이다.

In [6]:
import json

def evaluate_format_compliance(model, tokenizer, test_cases):
    """
    모델이 요청된 포맷(예: JSON)을 준수하는지 테스트하는 함수다.
    """
    results = []

    for case in test_cases:
        prompt = case["prompt"]
        expected_format = case.get("expected_format", "text")

        # 모델 추론
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=200)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 프롬프트 제외한 응답만 추출 (간단한 처리)
        response_content = response[len(prompt):].strip()

        # 검증 로직
        is_valid = False
        if expected_format == "json":
            try:
                json.loads(response_content)
                is_valid = True
            except:
                is_valid = False

        results.append({
            "prompt": prompt,
            "response": response_content,
            "valid_format": is_valid
        })

    return results

# 테스트 케이스
tests = [
    {
        "prompt": "사과의 영양성분을 JSON 형식으로 알려줘.\n",
        "expected_format": "json"
    }
]

# (실제 실행 시 모델 로드 필요)
# results = evaluate_format_compliance(model, tokenizer, tests)

## 5. 재앙적 망각(Catastrophic Forgetting) 테스트

도메인 지식을 배우느라 기본적인 상식이나 언어 능력을 잃어버렸는지 확인하는 과정이다. 간단한 상식 퀴즈를 통해 이를 감지할 수 있다.

In [7]:
def test_general_knowledge(model, tokenizer):
    """
    기본 상식 질문을 통해 재앙적 망각 여부를 진단한다.
    """
    basic_questions = [
        {"q": "1 + 1은 무엇인가?", "a": ["2", "이"]},
        {"q": "대한민국의 수도는 어디인가?", "a": ["서울"]},
        {"q": "사과는 무슨 색인가?", "a": ["빨간", "초록", "red", "green"]}
    ]

    correct_count = 0

    print("\n[재앙적 망각 테스트]")
    for item in basic_questions:
        inputs = tokenizer(item['q'], return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=20)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 정답 키워드가 포함되었는지 확인
        is_correct = any(ans in response for ans in item['a'])
        if is_correct:
            correct_count += 1
            print(f"질문: {item['q']} -> 정답 (응답: {response.strip()})")
        else:
            print(f"질문: {item['q']} -> 오답 (응답: {response.strip()})")

    score = correct_count / len(basic_questions) * 100
    print(f"\n기본 지식 보존율: {score:.1f}%")

    if score < 80:
        print("경고: 재앙적 망각이 의심된다. 학습률을 낮추거나 데이터 믹싱을 고려해야 한다.")

test_general_knowledge(model, tokenizer)


[재앙적 망각 테스트]
질문: 1 + 1은 무엇인가? -> 정답 (응답: 1 + 1은 무엇인가? 1 + 1은 2입니다. 이는 기본적인 덧셈의 규칙)
질문: 대한민국의 수도는 어디인가? -> 정답 (응답: 대한민국의 수도는 어디인가? 대한민국의 수도는 서울이다. 서울은 수도권에 위치하며, 대한민국)
질문: 사과는 무슨 색인가? -> 오답 (응답: 사과는 무슨 색인가? 사과의 색깔은 무엇인가요?

사과는 일반적으로 **빨강)

기본 지식 보존율: 66.7%
경고: 재앙적 망각이 의심된다. 학습률을 낮추거나 데이터 믹싱을 고려해야 한다.


## 6. 요약

이 챕터에서는 학습된 모델을 다각도로 평가하는 방법을 실습했다.

1. **자동 지표**: PPL, BLEU 등으로 학습 상태를 빠르게 진단한다.
2. **LLM-as-Judge**: GPT-4 등을 활용해 사람처럼 정성적인 평가를 수행한다.
3. **기능 검증**: JSON 출력 등 지시 이행 능력을 테스트한다.
4. **망각 방지**: 기본 상식 테스트를 통해 모델의 일반화 능력이 유지되는지 확인한다.

평가까지 마쳤다면, 이제 이 모델을 실제 서비스에 사용할 수 있도록 패키징하는 단계가 남았다.

다음 챕터는 **Chapter 14: 모델 병합 및 내보내기**로, LoRA 어댑터를 베이스 모델과 합치고 GGUF 등으로 변환하여 배포하는 방법을 다룬다.